In [1]:
!pip install sparqlwrapper

In [35]:
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# SPARQL Query
query = """
SELECT DISTINCT ?film ?filmLabel ?image WHERE {
  ?film wdt:P31 wd:Q11424;
   wdt:P154 ?image.
  FILTER (!CONTAINS(LCASE(STR(?image)), ".svg"))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 100
"""

def get_results(endpoint_url, query):
    """ Exécute la requête SPARQL et retourne les résultats en JSON """
    try:
        user_agent = "WDQS-example Python/%s.%s" % (
            sys.version_info[0],
            sys.version_info[1],
        )
        sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()
    except Exception as e:
        print(f"❌ Erreur lors de la requête SPARQL : {e}")
        return None

# Exécuter la requête
results = get_results(endpoint_url, query)

# Stocker les résultats dans un tableau
if results:
    data = []
    for result in results["results"]["bindings"]:
        film = result["film"]["value"] if "film" in result else "N/A"
        label = result["label"]["value"] if "label" in result else "N/A"
        image = result["image"]["value"] if "image" in result else "N/A"
        data.append((film, label, image))


import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?film ?filmLabel ?image WHERE {
  ?film wdt:P31 wd:Q11424;
   wdt:P154 ?image.
  FILTER (!CONTAINS(LCASE(STR(?image)), ".svg"))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["film"]["value"],
            result["filmLabel"]["value"],
            result["image"]["value"],
        )
    )

In [36]:
dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)
dataframe

,ville,pays,image
0,http://www.wikidata.org/entity/Q19866947,Le Cercle : Rings,http://commons.wikimedia.org/wiki/Special:File...
1,http://www.wikidata.org/entity/Q21028668,After Image,http://commons.wikimedia.org/wiki/Special:File...
2,http://www.wikidata.org/entity/Q13534614,Q13534614,http://commons.wikimedia.org/wiki/Special:File...
3,http://www.wikidata.org/entity/Q827105,Delicatessen,http://commons.wikimedia.org/wiki/Special:File...
4,http://www.wikidata.org/entity/Q126183,Working Girl,http://commons.wikimedia.org/wiki/Special:File...
...,...,...,...
95,http://www.wikidata.org/entity/Q19899248,Q19899248,http://commons.wikimedia.org/wiki/Special:File...
96,http://www.wikidata.org/entity/Q2702180,"Coffy, la panthère noire de Harlem",http://commons.wikimedia.org/wiki/Special:File...
97,http://www.wikidata.org/entity/Q1070554,Stuart Little 2,http://commons.wikimedia.org/wiki/Special:File...
98,http://www.wikidata.org/entity/Q846679,Karaté Kid,http://commons.wikimedia.org/wiki/Special:File...


In [37]:
import requests
import shutil
import os

def download_image(url, save_path="images"):
    """
    Télécharge une image depuis une URL et l'enregistre dans le dossier spécifié.
    
    :param url: URL de l'image à télécharger
    :param save_path: Dossier où enregistrer l'image (par défaut "images")
    :return: Chemin du fichier téléchargé si succès, None sinon
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    
    # Requête pour récupérer l'image
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)

    if request.status_code == 200:
        # Crée le dossier s'il n'existe pas
        os.makedirs(save_path, exist_ok=True)

        # Récupère le nom de fichier depuis l'URL
        filename = os.path.basename(url)
        file_path = os.path.join(save_path, filename)

        # Sauvegarde l'image
        with open(file_path, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
        
        print(f"✅ Image téléchargée : {file_path}")
        return file_path  # Retourne le chemin de l'image sauvegardée

    print(f"❌ Échec du téléchargement : {url}")
    return None  # Retourne None en cas d'échec



In [38]:

# Télécharger les images et stocker les chemins dans une nouvelle colonne
dataframe["downloaded_image"] = dataframe["image"].apply(lambda url: download_image(url))



✅ Image téléchargée : images\RIngsLogo.png
✅ Image téléchargée : images\After%20Image%202001.jpg
✅ Image téléchargée : images\Salinger%20Documentary%20Logo.jpg
✅ Image téléchargée : images\Delicatessen%20movie%20logo.png
✅ Image téléchargée : images\Working%20Girl.jpg
✅ Image téléchargée : images\The%20Silence%20of%20the%20Lambs.png
✅ Image téléchargée : images\Viridiana%20movie%20black%20logo.png
✅ Image téléchargée : images\Ghost-World.png
✅ Image téléchargée : images\Pink%20Panther%20logo-en.jpg
✅ Image téléchargée : images\The%20SpongeBob%20Movie%20Sponge%20Out%20of%20Water%20logo.png
✅ Image téléchargée : images\Fight%20Club%20logo1.png
✅ Image téléchargée : images\Predator%20logo.jpg
✅ Image téléchargée : images\D9%20Logo.jpg
✅ Image téléchargée : images\The%20Runaways%20logo.jpg
✅ Image téléchargée : images\On%20the%20Waterfront%20%28logo%29.png
✅ Image téléchargée : images\Open%20season%20logo.PNG
✅ Image téléchargée : images\The%20SpongeBob%20Movie-%20Sponge%20on%20the%20Run%2

In [39]:

def count_files_in_folder(folder_path):
    try:
        return len(os.listdir(folder_path))  # Compte tous les fichiers et dossiers
    except FileNotFoundError:
        print("❌ Le dossier n'existe pas.")
        return 0

# Exemple d'utilisation
folder = "images"  # Remplace par ton dossier
count = count_files_in_folder(folder)
print(f"📂 Le dossier '{folder}' contient {count} éléments.")


📂 Le dossier 'images' contient 99 éléments.


In [41]:
pip install pillow exifread


  Obtaining dependency information for exifread from https://files.pythonhosted.org/packages/db/d6/189b0016ae8995ad94cd6e2573baf0c289ff862996821d4e42eb6a0206e3/ExifRead-3.0.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/40.4 kB ? eta -:--:--
   ---------- ----------------------------- 10.2/40.4 kB ? eta -:--:--
   ---------------------------------------- 40.4/40.4 kB 642.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
